## 2D Boussinesq (buoyancy-driven) flow example

### equations solved:
conservation of momentum (inviscid and incompressible form of Navier-Stokes equation)
$$
\partial_t v+\mathrm{\nabla}\cdot\left(\vec{v}\otimes \vec{v}\right)=-\mathrm{\nabla\pi}-\vec{g}\frac{\theta^\prime}{\theta_0}
$$
conservation of energy
$$
\partial_t \theta+ \mathrm{\nabla}\cdot\left(\vec{v}\theta\right)=0
$$
continuity equation(conservation of mass)
$$
\mathrm{\nabla}\cdot\ \vec{v}=0
$$

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PyMPDATA-examples')

In [2]:
import os
#os.environ["NUMBA_DISABLE_JIT"]="1"

import warnings
from collections import namedtuple

from numba import jit
from numba.core.errors import NumbaExperimentalFeatureWarning

import numpy as np
from matplotlib import colors, pyplot
import ipywidgets
from IPython.display import display

from open_atmos_jupyter_utils import show_plot
from open_atmos_jupyter_utils.show_anim import show_anim

from PyMPDATA import Options, ScalarField, VectorField, Solver, Stepper
from PyMPDATA.boundary_conditions import Periodic
from PyMPDATA.impl.enumerations import IMPL_META_AND_DATA, META_AND_DATA_DATA, IMPL_BC

## Parameters

In [3]:
OPTIONS = Options(n_iters=2, infinite_gauge=True, nonoscillatory=True)

SETUP = {
    "outfreq": 1,
    "N": 20,
    "M": 20,
    "Tht_ref": 300.,
    "Tht_dlt": .5,
    "g": 9.81,
    "r0": 250.,
    "x0": 1000,
    "y0": 260,
    "dt": 7.5,
    "prs_tol": 1e-7,
    "k_iters": 4,
}

SETUP["nt"] = int(600//SETUP["dt"])
SETUP["err_tol"] = SETUP["prs_tol"] / SETUP["dt"]
SETUP["grid"] = SETUP["N"], SETUP["M"]
SETUP["dxy"] = (2000/SETUP["N"], 2000/SETUP["M"])
SETUP["halo"] = OPTIONS.n_halo
SETUP["grid_with_halo"] = SETUP["N"] + SETUP["halo"]*2, SETUP["M"] + SETUP["halo"]*2

SETUP = namedtuple("Setup", SETUP.keys())(**SETUP)

assert SETUP.dt * SETUP.nt == 600
type(SETUP)

__main__.Setup

## initial condition

In [4]:
xi, yi = np.indices(SETUP.grid)

mesh = np.full(SETUP.grid, fill_value=SETUP.Tht_ref, dtype=float)
mesh += np.where(
    np.sqrt(
        ((xi+1/2) * SETUP.dxy[0] - SETUP.x0)**2 +
        ((yi+1/2) * SETUP.dxy[1] - SETUP.y0)**2
    ) < SETUP.r0,
    SETUP.Tht_dlt,
    0
)

## PyMPDATA setup

In [5]:
_U = 0
_W = 1
_T = 2

bcond = (Periodic(), Periodic())
field_ctor_kwargs = {'halo': SETUP.halo, 'boundary_conditions': bcond}

advector = VectorField(data=(
    np.zeros((SETUP.N+1, SETUP.M)),
    np.zeros((SETUP.N, SETUP.M+1))
), **field_ctor_kwargs)
stepper = Stepper(options=OPTIONS, grid=SETUP.grid, n_threads = 1)
advector.assemble(stepper.traversals)

def new_sf(g):
    field = ScalarField(data=np.zeros(g), **field_ctor_kwargs)
    field.assemble(stepper.traversals)
    return field

solver_ctor_kwargs = {'stepper': stepper, 'advector': advector}
solvers = {
    _U: Solver(advectee=new_sf(SETUP.grid), **solver_ctor_kwargs),
    _W: Solver(advectee=new_sf(SETUP.grid), **solver_ctor_kwargs),
    _T: Solver(advectee=ScalarField(data=mesh, **field_ctor_kwargs), **solver_ctor_kwargs),
}

ADVECTEE_DATA = tuple(
    solvers[idx].advectee.data for idx in range(3)
)

ADVECTOR_DATA = tuple(
    datum for datum in advector.data
)

ARRAYS = {
    "rhs_w": np.zeros(SETUP.grid),
    "stash": np.zeros((len(SETUP.grid), *SETUP.grid_with_halo)),
    "vip_rhs": np.zeros((len(SETUP.grid), *SETUP.grid_with_halo)),
    "Phi": np.zeros(SETUP.grid_with_halo),
    "tmp_uvw": np.zeros((len(SETUP.grid), *SETUP.grid_with_halo)),
    "lap_tmp": np.zeros((len(SETUP.grid), *SETUP.grid_with_halo)),
    "lap_err": np.zeros(SETUP.grid),
    "err": np.zeros(SETUP.grid_with_halo),
    "p_err": np.zeros((SETUP.k_iters, *SETUP.grid_with_halo)),
    "lap_p_err": np.zeros((SETUP.k_iters, *SETUP.grid)),
}
ARRAYS = namedtuple("Arrays", ARRAYS.keys())(**ARRAYS)

IDX = {
    "interior": (slice(SETUP.halo, -SETUP.halo), slice(SETUP.halo, -SETUP.halo)),
    "grad_left": (
        (slice(SETUP.halo + 1, None if SETUP.halo==1 else (-SETUP.halo+1)), slice(SETUP.halo,-SETUP.halo)),
        (slice(SETUP.halo,-SETUP.halo), slice(SETUP.halo+1,None if SETUP.halo==1 else (-SETUP.halo+1)))
    ),
    "grad_right": (
        (slice(None if SETUP.halo==1 else (SETUP.halo-1),-SETUP.halo-1), slice(SETUP.halo,-SETUP.halo)),
        (slice(SETUP.halo,-SETUP.halo), slice(None if SETUP.halo==1 else (SETUP.halo-1), -SETUP.halo-1))
    ),
    "intrp_left": ( 
        (slice(None, -1), slice(None, None)),
        (slice(None, None), slice(None, -1)),
    ),
    "intrp_right": ( 
        (slice(1, None), slice(None, None)),
        (slice(None, None), slice(1, None)),
    )
}
IDX = namedtuple("Idx", IDX.keys())(**IDX)

## JIT-compilable functions

In [1]:
fill_halos_scalar = stepper.traversals._code["fill_halos_scalar"]

@jit(**OPTIONS.jit_flags)
def grad(psi, axis):
    return (psi[IDX.grad_left[axis]] - psi[IDX.grad_right[axis]]) / 2 / SETUP.dxy[axis]
    
@jit(**OPTIONS.jit_flags)
def calc_grad_Phi(arg, phi):
    for k in (0,1):
        arg[k][IDX.interior] = grad(phi, axis=k)

@jit(**OPTIONS.jit_flags)
def data(impl):
    return impl[IMPL_META_AND_DATA][META_AND_DATA_DATA]

@jit(**OPTIONS.jit_flags)
def pressure_solver_loop_body(converged):
    tmp_den = [1.] * SETUP.k_iters
    alpha = [1.] * SETUP.k_iters
    for v in range(0, SETUP.k_iters):
        tmp_den[v] = np.sum(ARRAYS.lap_p_err[v]**2)
        if tmp_den[v] != 0:
            beta = - np.dot(
                ARRAYS.err[IDX.interior].ravel(),
                ARRAYS.lap_p_err[v].ravel()
            ) / tmp_den[v]
        else:
            beta = .25
        ARRAYS.Phi[IDX.interior][:] += beta * ARRAYS.p_err[v][IDX.interior]
        ARRAYS.err[IDX.interior][:] += beta * ARRAYS.lap_p_err[v]

        error = max(
            abs(np.amax(ARRAYS.err[IDX.interior])),
            abs(np.amin(ARRAYS.err[IDX.interior]))
        )

        if error <= SETUP.err_tol: 
            converged = True

        ARRAYS.lap_err[:] = lap(ARRAYS.err,False)

        for l in range(v):
            if tmp_den[l] != 0:
                alpha[l] = - np.dot(ARRAYS.lap_err.ravel(), ARRAYS.lap_p_err[l].ravel()) / tmp_den[l]
        if v < SETUP.k_iters - 1:
            ARRAYS.p_err[v + 1][IDX.interior][:] = ARRAYS.err[IDX.interior]
            ARRAYS.lap_p_err[v + 1][:] = ARRAYS.lap_err
            for l in range(v):
                ARRAYS.p_err[v + 1][IDX.interior][:] += alpha[l] * ARRAYS.p_err[l][IDX.interior]
                ARRAYS.lap_p_err[v + 1][:] += alpha[l] * ARRAYS.lap_p_err[l]
        else:
            ARRAYS.p_err[0][IDX.interior][:] = ARRAYS.err[IDX.interior][:] + alpha[0] * ARRAYS.p_err[0][IDX.interior]
            ARRAYS.lap_p_err[0][:] = ARRAYS.lap_err[:] + alpha[0] * ARRAYS.lap_p_err[0]
            for l in range(1, v+1):
                ARRAYS.p_err[0][IDX.interior][:] += alpha[l] * ARRAYS.p_err[l][IDX.interior]
                ARRAYS.lap_p_err[0][:] += alpha[l] * ARRAYS.lap_p_err[l]
    return converged

@jit(**OPTIONS.jit_flags)
def calc_gc_interpolate_in_space():
    for axis in (_U, _W):
        ADVECTOR_DATA[axis][:] = SETUP.dt / SETUP.dxy[axis] * (
            (
                ARRAYS.stash[axis][IDX.intrp_right[axis]] - 
                ARRAYS.stash[axis][IDX.intrp_left[axis]]
            ) / 2 + ARRAYS.stash[axis][IDX.intrp_left[axis]]
        )

@jit(**OPTIONS.jit_flags)
def ini_pressure():
    for k in (_U, _W):
        ARRAYS.Phi[IDX.interior][:] -= 0.5 * np.power(ADVECTEE_DATA[k][IDX.interior][:], 2)
    ARRAYS.Phi[IDX.interior][:] -= np.sum(ARRAYS.Phi[IDX.interior]) / (SETUP.N * SETUP.M)

@jit(**OPTIONS.jit_flags)
def lap(Phi, err_init): 
    xchng_pres_old(Phi)
    calc_grad_Phi(ARRAYS.lap_tmp, Phi)

    if err_init:
        for k in (_U, _W):
            ARRAYS.lap_tmp[k][:] -= ARRAYS.tmp_uvw[k]

    for k in (_U, _W):
        xchng_pres_old(ARRAYS.lap_tmp[k])
        
    return (
        grad(ARRAYS.lap_tmp[0], axis=0) +
        grad(ARRAYS.lap_tmp[1], axis=1)
    )

@jit(**OPTIONS.jit_flags)
def pressure_solver_loop_init():
    ARRAYS.p_err[0][IDX.interior][:] = ARRAYS.err[IDX.interior]
    ARRAYS.lap_p_err[0][:] = lap(ARRAYS.p_err[0], False)

@jit(**OPTIONS.jit_flags)
def pressure_solver_update():
    for k in (_U, _W):
        ARRAYS.tmp_uvw[k][:] = ADVECTEE_DATA[k]
    ARRAYS.err[IDX.interior][:] = lap(ARRAYS.Phi, True)
    pressure_solver_loop_init()

    iters, converged = 0, False
    while not converged:        
        converged = pressure_solver_loop_body(converged)
        iters += 1
        if iters > 10000:
            raise ArithmeticError("stuck in pressure solver")
             
    xchng_pres_old(ARRAYS.Phi)
    calc_grad_Phi(ARRAYS.tmp_uvw, ARRAYS.Phi)

@jit(**OPTIONS.jit_flags)
def xchng_pres(impl):
    fill_halos_scalar(0, *impl[IMPL_META_AND_DATA], impl[IMPL_BC], None)

@jit(**OPTIONS.jit_flags)
def calc_gc_extrapolate_in_time():
    for k in (_U, _W):
        ARRAYS.stash[k][IDX.interior] = -.5 * ARRAYS.stash[k][IDX.interior] + 1.5 * ADVECTEE_DATA[k][IDX.interior]
        ARRAYS.stash[k][IDX.interior] = -.5 * ARRAYS.stash[k][IDX.interior] + 1.5 * ADVECTEE_DATA[k][IDX.interior]
        xchng_pres_old(ARRAYS.stash[k])

@jit(**OPTIONS.jit_flags)
def fill_stash():
    for k in (_U, _W):
        ARRAYS.stash[k][IDX.interior] = ADVECTEE_DATA[k][IDX.interior]
        xchng_pres_old(ARRAYS.stash[k])
        
@jit(**OPTIONS.jit_flags)
def xchng_pres_old(data):
    data[0:SETUP.halo, :] = data[-2*SETUP.halo:-SETUP.halo,:]
    data[-SETUP.halo:, :] = data[SETUP.halo:2*SETUP.halo, :]
    data[:,0:SETUP.halo] = data[:,-2*SETUP.halo : -SETUP.halo]
    data[:,-SETUP.halo:] = data[:,SETUP.halo : 2*SETUP.halo]

@jit(**OPTIONS.jit_flags)
def vip_rhs_apply():
    for k in (_U, _W):
        ADVECTEE_DATA[k][:] += 0.5 * SETUP.dt * ARRAYS.vip_rhs[k]
        ARRAYS.vip_rhs[k][:] = 0

@jit(**OPTIONS.jit_flags)
def apply_half_rhs():              
    ADVECTEE_DATA[_W][IDX.interior] += ARRAYS.rhs_w * SETUP.dt/2

@jit(**OPTIONS.jit_flags)
def finilize_rhs():
    for k in (_U, _W):
        ARRAYS.vip_rhs[k][:] = -ADVECTEE_DATA[k]

@jit(**OPTIONS.jit_flags)
def update_rhs():
    ARRAYS.rhs_w[:] = SETUP.g * (ADVECTEE_DATA[_T][IDX.interior] - SETUP.Tht_ref) / SETUP.Tht_ref

@jit(**OPTIONS.jit_flags)
def pressure_solver_apply():
    for k in (_U, _W):
        ADVECTEE_DATA[k][:] -= ARRAYS.tmp_uvw[k]
        ARRAYS.vip_rhs[k][:] += ADVECTEE_DATA[k]
        ARRAYS.vip_rhs[k][:] /= 0.5 * SETUP.dt

NameError: name 'stepper' is not defined

## Simulation loop
Progress bar stays at 0 until all functions compile

In [ ]:
#%%time
progress = ipywidgets.FloatProgress(max=SETUP.nt)
display(progress)

output = np.empty((SETUP.nt//SETUP.outfreq+1, *SETUP.grid))
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore",category= NumbaExperimentalFeatureWarning)
    pressure_solver_update()
    xchng_pres_old(ARRAYS.Phi)
    calc_grad_Phi(ARRAYS.tmp_uvw, ARRAYS.Phi)
    pressure_solver_apply()
    ini_pressure()
    xchng_pres_old(ARRAYS.Phi)
    calc_grad_Phi(ARRAYS.tmp_uvw, ARRAYS.Phi)
    for k in (0,1):
        ARRAYS.vip_rhs[k][:] -= ARRAYS.tmp_uvw[k]
    for step in range(SETUP.nt + 1):
        if step != 0:
            calc_gc_extrapolate_in_time() 
            calc_gc_interpolate_in_space() 
            fill_stash()
            apply_half_rhs()
            vip_rhs_apply()
            for solver in solvers.values():
                solver.advance(n_steps=1)
            update_rhs()
            apply_half_rhs()
            finilize_rhs()
            pressure_solver_update()
            pressure_solver_apply()
        if step % SETUP.outfreq == 0:
            output[step] = solvers[_T].advectee.get().copy()
        progress.value = step
with open("output.npy", 'wb') as f:
    np.save(f, np.array(output))

FloatProgress(value=0.0, max=80.0)

## Visualisation

In [ ]:
freq = int(10)
progress = ipywidgets.FloatProgress(max=len(output)/freq)

def plot(step,plotflag=False):
    data = output[step]
    grid = data.shape
    x, y = np.indices(grid)
    colorticks = np.linspace(299.95, 300.55, 7, endpoint=True)
    cmap = colors.ListedColormap(["white", "purple", "blue", "green","orange","yellow"])

    fig = pyplot.figure(figsize=(5,6))
    pyplot.title("t/dt="+str(step))
    pyplot.xlabel("x/dx")
    pyplot.ylabel("y/dy")
    pyplot.contour(x+1/2, y+1/2, data, levels=colorticks[1:], colors='k',linewidths=0.3)
    pyplot.imshow(data.T, origin="lower", extent=(0, grid[0], 0, grid[1]),
                  cmap=cmap, vmin=colorticks[0], vmax=colorticks[-1])
    pyplot.grid()
    cax = pyplot.axes([0.95, 0.262, 0.05,0.466])
    if plotflag:
        pyplot.colorbar(ticks=colorticks,label = "Temperature[K]",cax = cax)
    else:
        pyplot.colorbar(ticks=colorticks,label = "Temperature[K]")
    progress.value += 1
    return fig
    
display(progress)
show_anim(plot, frame_range=range(len(output))[::freq])

In [ ]:
show_plot(fig=plot(0,True))


In [ ]:
show_plot(fig=plot(800,True))